In [1]:
from sage.modules.free_module_element import vector
from sage.matrix.constructor import matrix
from sage.rings.infinity import Infinity
from sage.matrix.special import block_matrix
from sage.matrix.special import diagonal_matrix
from sage.matrix.special import zero_matrix
from sage.matrix.special import identity_matrix
from sage.functions.other import floor, ceil
from sage.numerical.mip import MixedIntegerLinearProgram
from sage.structure.sage_object import SageObject
from sage.interfaces.four_ti_2 import four_ti_2
from sage.numerical.mip import MIPSolverException

from itertools import combinations_with_replacement

from math import log

import itertools
import logging
import time
#import GS_timing as timing
import sys
import numpy as np


#global variable declaration here

global n
global m
global B
global p
global EPS
global tau
global vote_count
global lamda
global max_V_i
global min_bcost
global max_bcost
global F_container
global r
global S_star
global rnd_seed
global loop_counter
global F_count
global F_signal



class construct_Gamma_iterator:
    '''iterator that yields numbers in Gamma'''
    def __init__(self,u,l,n,base=2):
        
        #make the difference upper-lower bounds
        difference = []
        for i in range(n):
            dif = (u[i]-l[i])
            difference.append(dif)
        
        #choose the biggest element of the difference
        max_val = float('-inf')
        for dif in difference:
            max_entry = dif.norm(float('inf'))
            if max_entry>max_val:
                max_val = max_entry
        
        
        self.logarithm = floor(log(max_val,2)) #floor nebo ceil?
        
        #counters and last item
        self.num_of_iterations = 0
        self.last = 1
        self.base = base

    def __iter__(self):
        return self

    def __next__(self):
        last_gamma = self.last
        self.num_of_iterations = self.num_of_iterations +1
        
        if self.num_of_iterations > self.logarithm:
            
            raise StopIteration
        self.last = last_gamma * self.base
        
        return int(last_gamma)

def sage_int_vector_to_tuple(vec):
    return tuple((int(ri) for ri in vec))

def sage_int_matrix_to_list_of_tuples(M):
    # Converts a sage integer matrix to a list of rows,
    # each row being a tuple of ints
    # thus matrix(rs) would return M
    rs = M.rows()
    rs= list(sage_int_vector_to_tuple(r) for r in rs)
    return rs

class NFoldIP(SageObject):
    
    def __init__(self, A, D, n, b, l, u, w, verbose = logging.ERROR, graver_complexity = "exact", current_solution = None,
                 experimental=False, instancename="instancename", gamma="log2",solver="GLPK", milp_timelimit=.05, augip_timelimit=.05):
        
        #SET LOGGING
        self.NFoldLogging = logging.getLogger(__name__)
        if not getattr(self.NFoldLogging, 'handler_set', None):
            formatter = logging.Formatter(fmt='%(asctime)s.%(msecs)03d  %(message)s',datefmt='%H:%M:%S') #formates time for logging
            handler1 = logging.StreamHandler() #stdout
            handler1.setFormatter(formatter)
            if(verbose!="None"):
                handler1.setLevel(verbose)
                self.NFoldLogging.setLevel(verbose)
            self.NFoldLogging.addHandler(handler1)
            self.NFoldLogging.handler_set = True
            if(verbose=="None"):
                self.NFoldLogging.disabled = True
        self.instancename = instancename
        self.filelog = open(self.instancename+'.log', 'w')
        
        
        self.solver = solver
        
        #CHECK THE VALIDITY OF GIVEN DATA
        self._check_validity_of_data(A,D,n,b,l,u,w,current_solution)
        
        #MATRICES AND VECTORS OF THE GIVEN PROBLEM
        self.A = A #matrix A
        self.D = D #matrix D
        self.n = n
        self.b = b
        self.l = l #lower bound
        self.u = u #upper bound
        self.w = w #objective function
        self.w_vector = vector(sum((ww.list() for ww in w), []))
        self.t = D.ncols() #number of columns of A,D
        self.s = A.nrows() #number of rows of A
        self.r = D.nrows() #number of rows of D
        
        
        self.fulllog = {}
        self.fulllog["inst"] = dict(zip(("A","D","n","t","s","r","current_solution"),(A,D,n,self.t,self.s,self.r,current_solution)))
        self.fulllog["inst"]["A"] = sage_int_matrix_to_list_of_tuples(self.fulllog["inst"]["A"])
        self.fulllog["inst"]["D"] = sage_int_matrix_to_list_of_tuples(self.fulllog["inst"]["D"])
        self.fulllog["inst"]["current_solution"] = [sage_int_vector_to_tuple(brick) for brick in current_solution]
        self.fulllog["inst"]["b"] = [sage_int_vector_to_tuple(brick) for brick in b]
        self.fulllog["inst"]["l"] = [sage_int_vector_to_tuple(brick) for brick in l]
        self.fulllog["inst"]["u"] = [sage_int_vector_to_tuple(brick) for brick in u]
        self.fulllog["inst"]["w"] = [sage_int_vector_to_tuple(brick) for brick in w]
        self.fulllog["dimension"] = int(n*self.t)
        self.fulllog["Delta"] = int(max((A.numpy().max(), D.numpy().max())))
        self.fulllog["iterations"] = {}
        self.fulllog["gamma"] = str(gamma)
        self.fulllog["instancename"] = str(instancename)
        self.fulllog["milp_timelimit"] = int(milp_timelimit)
        self.fulllog["augip_timelimit"] = int(augip_timelimit)
       
        #INITAL FEASIBLE SOLUTION 
        self.current_solution = current_solution #can be None
        
        if experimental == True:
            self._find_good_step = self._find_good_step_experimental
            self.experimental = True
            self.GA = four_ti_2.graver(self.A) #Graver basis of A
        elif experimental == "ng1":
            self.experimental = "ng1"
            self._find_good_step = self._find_good_step_experimental_ng
        elif experimental == "nginfty":
            self.experimental = "nginfty"
            self._find_good_step = self._find_good_step_experimental_ng

        self.average_good_step_time = -1
        self.max_good_step_time = 0
        self.feasible_good_steps = 0
        
        self.gamma = gamma
        
        #COMPUTE/SET THE GRAVER COMPLEXITY
        if not experimental:
            self.GA = four_ti_2.graver(self.A) #Graver basis of A
        if graver_complexity == "exact":
            self.graver_complexity = self.exact_graver_complexity()
        elif graver_complexity =="approximate":
            self.graver_complexity = self.approximate_graver_complexity()
        elif type(graver_complexity) == int:
            self.graver_complexity = int(graver_complexity)
        else:
            print ("not a valid argument for the graver_complexity keyword")
        self.fulllog["graver_complexity"] = int(self.graver_complexity)
        


        if not experimental:
            #CONSTRUCT ZE
            self.ZE = self._construct_ZE() #construct ZE 
            self.experimental = False
        
        self.augip_initialized = False
        
        # TIMELIMITS
        self.milp_timelimit = milp_timelimit
        self.augip_timelimit = augip_timelimit

        #LOGGING  PROPERTIES
        self.graver_best_counter = 0		
        self.all_find_good_step_counter = 0		
        self.find_good_step_counter = 0		
        self.set_of_time_find_good_step = []
        
        #VARS FOR SOLUTION
        self.native_solution = None
        self.glpk_solution = None
        
        
        
    def _check_validity_of_data(self,A,D,n,b,l,u,w,current_solution):
        "Check if the sizes and bounds are valid."
        
        self.NFoldLogging.info('Checking the validity of input data.')    
        
        #sizes of A,D
        if D.ncols() != A.ncols():
            raise ValueError("Matrices A,D don't have the same number of columns! The instance was not initialized.")
        
        #vectors of lists of l/u bounds and current solution (if any)
        if len(l) != n:
            print("zde-----------")
            print (l)
            print (n)
            raise ValueError("The length of the lower bound vector must have the same number of vectors as the input n! The instance was not initialized.")
        if len(u) != n:
            raise ValueError("The length of the upper bound vector must have the same number of vectors as the input n! The instance was not initialized.")
        if current_solution != None:
            if len(current_solution) != n:
                raise ValueError("The length of the current solution vector must have the same number of vectors as the input n! The instance was not initialized.")
                
        #sizes of vectors in l/u bounds and current solution (if any)                 
        for i in range(n):    
            if len(l[i]) != D.ncols():
                raise ValueError("The length of the first vector in lower bounds must have the same lenght as the number of columns in D! The instance was not initialized.")
            if len(u[i]) != D.ncols():
                raise ValueError("The length of the first vector in upper bounds must have the same lenght as the number of columns in D! The instance was not initialized.")
            if current_solution != None:
                if len(current_solution[i]) != D.ncols():
                    raise ValueError("The length of the first vector in the current solution must have the same lenght as the number of columns in D! The instance was not initialized.")
            
        #upper/lower can't be infinity!
        for v in l:
            for vi in v:
                if v==float('inf'):
                    raise ValueError("Lower bound can't be infinity! The instance was not initialized.")
                if v==float('-inf'):
                    raise ValueError("Lower bound can't be -infinity! The instance was not initialized.")
        for v in u:
            for vi in v:
                if u==float('inf'):
                    raise ValueError("Upper bound can't be infinity! The instance was not initialized.")
                if u==float('-inf'):
                    raise ValueError("Upper bound can't be -infinity! The instance was not initialized.")
        
        #if upper<=lower in the first vector according to current solution (if any)
        for i in range(n):
            for j in range(D.ncols()):
                if l[i][j]> u[i][j]:
                     raise ValueError("Upper bounds must be greater or equal to the lower bounds! The instance was not initialized.")
                if current_solution != None:
                    if l[i][j] > current_solution[i][j]:
                        raise ValueError("Given current solution does not satisfy lower bounds! The instance was not initialized.")
                    if current_solution[i][j]>u[i][j]:
                        raise ValueError("Given current solution does not satisfy upper bounds! The instance was not initialized.")
                         
        #size of b
        if len(b) != n+1:
            raise ValueError("The length of b vector must have the same number of vectors as the input n + 1! The instance was not initialized.")
        if len(b[0]) != D.nrows():
            raise ValueError("The first vector of b must have the same size as the number of rows in D! The instance was not initialized.")
        for i in range(n):
            if len(b[i+1])!=A.nrows():
                raise ValueError("The second and next vectors of b must have the same size as the number of rows in A! The instance was not initialized.")          
            
        #size of w    
        if len(w) != n:
            raise ValueError("The length of the objective function w must have the same number of vectors as the input n! The instance was not initialized.")
        for i in range(n):
            if len(w[i]) != D.ncols():
                raise ValueError("The vectors of w must have the same size as the number of columns in D/A! The instance was not initialized.")
        
        self.NFoldLogging.info('Input data checked.')
        
        
    def exact_graver_complexity(self):
        """
        Return an exact value of the Graver complexity of the given data from the definition.
        """
        value = four_ti_2.graver((self.D)* self.GA.transpose()).norm(Infinity)
        self.NFoldLogging.info('Exact Graver complexity: ',value)
        return value
      
        
    def approximate_graver_complexity(self):
        """
        Return an approximate value of the Graver complexity of the given matrices.
        """
        p = self.GA.nrows() * 2
        
        G = self.GA.transpose()
        
        mult = self.D*G #ZDE BYL BUG! Bylo tu self.A*G
        
        biggestValue = mult.height()
        value = ceil(p*( (self.r* biggestValue)^self.r)) # A TADY BYLO (sqrt(self.r)) uvnitr
        self.NFoldLogging.info('Approximate Graver complexity: ',value)
        return value
        
        
    def _construct_ZE_slower(self):
        """
        Construct ZE - the sum of at most Graver complexity elements of the matrix D using itertools.
        """
        #start = time.time() 
        GA2 = []
        for i in self.GA:
            GA2.append(i)
            GA2.append(-i)
        
        #add zero-vector
        GA2.append(vector((0,)*self.t))
        
        R = set()
        for x in combinations_with_replacement(GA2,self.graver_complexity):
            vv=sum(vector(v) for v in x)
            vv.set_immutable()
            R.add(vv)
            
        end = time.time()
        #print end-start    
        
        self.NFoldLogging.info('COMPUTING ZE - ZE has {} unique vectors'.format(len(R)))
                
        return R
    
    def _construct_ZE(self):
        """
        Construct ZE - the sum of at most Graver complexity elements of the matrix A.
        """
        self.NFoldLogging.debug('COMPUTING ZE:')
        vector0 = vector((0,)*self.t)
        vector0.set_immutable()
        count = 0
        R = {vector0}
        for i in range(self.graver_complexity):
            R_new = set()    
            for x in R:
                count = count+len(R)*self.GA.nrows() #counter of iterations in this inner forcycle
                for y in self.GA:
                    z1 = x+y
                    z2 = x-y
                    z1.set_immutable()
                    z2.set_immutable()
                    if z1 not in R:
                        self.NFoldLogging.debug('new unique vector: {}'.format(z1))
                    R_new.add(z1)
                    if z2 not in R:
                        self.NFoldLogging.debug('new unique vector: {}'.format(z2))
                    R_new.add(z2)
            R = R.union(R_new)
        R.add(vector0)              
        self.NFoldLogging.info('COMPUTING ZE - ZE has {} unique vectors, but {} not necessarily unique vectors were computed'.format(len(R),self.graver_complexity*count))
        return R
                
        
    def _find_good_step(self,gamma):
        """
        Dynamic program runs exactly here. Finding the best path in a weighted acyclic diagraph of layers from ZE elements.
        
        """

        from copy import copy
        current = [(vector((0,)*self.t), [], 0)]
            
        for i in range(self.n):
            last = current
            current = []
                
            for h in self.ZE:
                best_val = float('inf') 
                best_path = []
                if (i == self.n-1) and (self.D*h != vector((0,)*self.r)):
                    continue
                for g, g_path, g_val in last:
                    hg = vector(h-g)
                    hg.set_immutable()
                    if not (hg in self.ZE):
                        continue
                        
                    help = self.current_solution[i] + (hg * gamma)
                    lbub = all((ll <= hh for ll, hh in zip(self.l[i],help) )) and all((uu >= hh for hh,uu in zip(help, self.u[i]) )) #when lower and 
                    if lbub:
                        new_value = (hg * self.w[i]) + g_val
                        if new_value < best_val:
                            best_val = new_value
                            best_pred = g
                            best_path = copy(g_path)
    
                v = vector(h)
                v.set_immutable()
                #if h unreachable, best_path has not been initialized
                if best_val != float('inf'):
                    best_path.append(h)
                    current.append((v, best_path, best_val)) #(tuple([v, best_path, best_val]))
            
        last = current
        
        min = float('inf')
        path = []
        for (g, g_path, g_val) in last:
            if g_val<min:
                min = g_val
                path = g_path
        
        expanded_path = [path[0]] + [path[i] - path[i-1] for i in range(1,self.n)]
         
        return expanded_path
            
        
    def find_graverbest_step(self):
        
        """
        Generate generator for Gamma, which is a set of gammas.
        Then for each gamma in Gamma count good steps (function find_good_step(gamma)) and then choose the one step with the biggest dot product with the subjective function w.
        """
        self.NFoldLogging.debug('FINDING THE GRAVER BEST STEP: ')
        # MAX run time hardcoded to be one hour.
        previous_time = 3600
        current_min = 0
        min_step = vector((0,)*(self.n*self.t))
        fl = self.fulllog["iterations"][self.iteration]["gammas"]
        
        if self.gamma == "unit":
            Gamma = [1]
        elif self.gamma == "best" and (self.experimental in ["ng1", "nginfty"]):
            Gamma = self.construct_best_Gamma()
        else:
            ddd = {"log2":2, "log5":5, "log10":10}
            # logarithmic Gamma = default

            Gamma = construct_Gamma_iterator(self.u, self.l, self.n, base=ddd[self.gamma])
            
        
        for gamma in (Gamma):
            
            self.NFoldLogging.warning('Finding good step for GAMMA: {}'.format(gamma))
            fl[gamma] = {}
            #start_time = timing.micros()
            good_step = self._find_good_step(gamma)
            
            #if (self.average_good_step_time == -1) or (gamma==1):
            #    good_step = self._find_good_step(gamma)
            #else:
            #    good_step = self._find_good_step(gamma, timelimit=min((3*self.max_good_step_time, 5)))
                #good_step = self._find_good_step(gamma)
            #end_time = timing.micros()
            #previous_time = float(end_time - start_time) / 10**6 # convert us -> s
            #self.NFoldLogging.warning('Finding good step took {}'.format(previous_time))
            self.NFoldLogging.debug('good_step from find graverbest: {}'.format(good_step))
            
            fl[gamma]["g"] = sage_int_vector_to_tuple(good_step)
            fl[gamma]["obj"] = int(self.current_obj + self.w_vector.dot_product(gamma*good_step))
            
            if self.w_vector.dot_product(good_step) >= 0:
                fl[gamma]["augmenting"] = False
                break
            else:
                self.feasible_good_steps += 1
                fgs = self.feasible_good_steps
                self.max_good_step_time = max((self.max_good_step_time, previous_time))
                if self.average_good_step_time == -1:
                    self.average_good_step_time = previous_time
                else:
                    self.average_good_step_time = max(( self.average_good_step_time*( (fgs-1) / float(fgs)) + previous_time*(1/float(fgs)), previous_time))
                self.NFoldLogging.warning('Updated average time {}'.format(self.average_good_step_time))
                self.NFoldLogging.warning('Updated max time {}'.format(self.max_good_step_time))
            
            old_gamma = gamma
            #look if it's possible to take bigger gamma for prolonging the good_step
            new_gamma = float('inf')
            
           
            #print("current_solution = {}".format(self.current_solution))
            #print("Gamma = {}".format(gamma))
            #print("good_step = {}".format(good_step))            
            #print("lower bound = {}".format(self.l))            
            #print("upper bound = {}".format(self.u)) 


            for i in range(len(good_step)):
                if good_step[i]!=0:
                    
                    ii = floor(i/self.t)
                    jj = floor(i%self.t)
                    
                    li = -(self.current_solution[ii][jj] - self.l[ii][jj])/good_step[i]
                    ui = (self.u[ii][jj]-self.current_solution[ii][jj])/good_step[i]
                    #take the bigger one
                    if ui>li:
                        bi = floor(ui)
                    else:
                        bi = floor(li)
                    #if new_gamma is too long, make it shorter
                    if new_gamma > bi:
                        new_gamma = bi
            
            if new_gamma != float('inf'):
                if gamma<new_gamma:
                    gamma = new_gamma         
                    self.NFoldLogging.warning('GAMMA CHANGED -- Finding good step for gamma: {}'.format(gamma)) 
                      
            
            self.NFoldLogging.debug('good_step with prolonging from find graverbest: {}'.format(good_step))
            self.find_good_step_counter += 1
            dot_product = self.w_vector.dot_product(gamma*good_step)
            self.NFoldLogging.info('for step length {} the best step has value {}'.format(gamma,dot_product))
            self.filelog.write("("+str(self.current_obj + dot_product) + ","+str(previous_time)+") ")
            
            fl[old_gamma]["augmenting"] = True
            fl[old_gamma]["prolonged_gamma"] = int(gamma)
            fl[old_gamma]["obj"] = int(self.current_obj + dot_product)
            
            if dot_product < current_min:
                
                self.NFoldLogging.warning('For gamma {} the improvement is {}'.format(gamma,dot_product))
                current_min =  dot_product
                min_step = gamma*good_step
                self.NFoldLogging.debug('New better dot product: current_min = {}, min_step = {}'.format(current_min,min_step))
            
        
        self.NFoldLogging.info('GRAVER BEST STEP has value {} and is = {}'.format(current_min,min_step))
        self.NFoldLogging.warning('in this call of find_graverbest_step the function _find good step was called {} times'.format(self.find_good_step_counter))                                
        self.all_find_good_step_counter = self.all_find_good_step_counter + self.find_good_step_counter		
        # We set it to one initially because on the last iteration, we do not add 1
        self.find_good_step_counter = 1
                   
        return min_step
        
    def construct_best_Gamma(self):
        
        Gamma = set()
        gc = self.graver_complexity
        for i in range(self.n):
            for j in range(self.t):
                uu = self.u[i][j]
                ll = self.l[i][j]
                xx = self.current_solution[i][j]
                #print "ll:",ll,"uu:",uu,"xx:",xx
                for k in range(1,gc+1):
                    gamma_1 = floor((uu-xx)/k)
                    gamma_2 = floor((xx-ll)/k)
                    Gamma.add(int(gamma_1))
                    Gamma.add(int(gamma_2))
        Gamma = list(Gamma)
        Gamma.sort()
        Gamma.remove(0)
        return Gamma

                
        
    def _find_good_step_experimental(self,gamma,timelimit=None):
        GA = [v for v in self.GA] 
        GA += [-v for v in GA] 
        GA += [vector((0,)*self.t)]
        DGA = [self.D*v for v in GA]
        p = MixedIntegerLinearProgram(maximization=False, solver = "GLPK")
        # indexed as x[layer, sublayer, index of element in GA]
        # NEW indexed as x[layer, index of element in GA]
        # number of layers is n
        # OLD number of sublayers is graver_complexity
        x = p.new_variable(integer=True, nonnegative=True)
        
        
        # at most graver_complexity elements selected across all layers
        p.add_constraint(sum((x[i,k] for i in range(self.n) for k in range(len(GA)))) <= self.graver_complexity)
        #p.add_constraint(-self.graver_complexity <=  sum((x[i,k] for i in range(self.n) for k in range(len(GA)))))
        
        # the sum of selected elements is in Ker(D)
        p.add_constraint(sum((x[i,k]*DGA[k] for i in range(self.n) for k in range(len(GA)))) == 0)
        
        # lower and upper bounds
        for i in range(self.n):
            p.add_constraint(sum((gamma*x[i,k]*GA[k] for k in range(len(GA)))) + self.current_solution[i] <= self.u[i])
            p.add_constraint(sum((gamma*x[i,k]*GA[k] for k in range(len(GA)))) + self.current_solution[i] >= self.l[i])
        
        # minimize the objective
        f = 0
        for i in range(self.n):
            for k in range(len(GA)):
                f += x[i,k]*self.w[i].dot_product(GA[k])
        p.set_objective(f)
        
        if timelimit:
            p.solver_parameter("timelimit", timelimit)
        
        try:
            p.solve()
        except MIPSolverException:
            return vector((0,)*(self.n*self.t))
        if p.get_objective_value() > -1:
            return vector((0,)*(self.n*self.t))
        
        # Construct the augmenting step
        h = []
        for i in range(self.n):
            h.append(sum((int(p.get_values(x[i,k]))*GA[k] for k in range(len(GA)))))
        hh = []
        for hhh in h:
            hh += hhh
        hh = vector(hh)
    
        return hh
        
    def _find_good_step_experimental_ng(self,gamma,timelimit=.05):
        # norm can be "infty" or "1"; we constraint the solution to be bounded by self.graver_complexity of this norm
        # l_\infty
        #p = MixedIntegerLinearProgram(maximization=False, solver = solver)
        if not self.augip_initialized:
            #start = timing.micros()
            p = MixedIntegerLinearProgram(maximization=False, solver = self.solver)
            x = p.new_variable(integer=True, nonnegative=False)
            
            for i in range(self.n):
                for j in range(self.t):
                    ub = floor( min(self.graver_complexity*gamma, self.u[i][j] - self.current_solution[i][j]) / gamma)
                    lb = ceil( max(-self.graver_complexity*gamma, self.l[i][j] - self.current_solution[i][j]) / gamma)
                    #print ("lb is {} and ub is {}".format(lb, ub))
                    p.set_max(x[i,j],ub)
                    p.set_min(x[i,j],lb)
                    
                    
            
            An = self.create_An_matrix()
            
            for row in An:
                p.add_constraint(sum((row[i*self.t + j]*x[i,j] for j in range(self.t) for i in range(self.n))) == 0)
            
            if self.experimental=="ng1":
                pos = p.new_variable(integer=True, nonnegative=True)
                neg = p.new_variable(integer=True, nonnegative=True)
                for i in range(self.n):
                    for j in range(self.t):
                        
                        p.add_constraint(x[i,j] == pos[i,j] - neg[i,j])
                p.add_constraint( sum(pos[i,j] + neg[i,j] for i in range(self.n) for j in range(self.t)) <= self.graver_complexity )
            
            
            # minimize the objective
            f = 0
            for i in range(self.n):
                for j in range(self.t):
                    f += self.w[i][j]*x[i,j]
                    
            p.set_objective(f)
            self.augip = {}
            self.augip["p"] = p
            self.augip["x"] = x
            #end = timing.micros()
            #self.fulllog["augip_init_time"] = float( (end-start)/10**6)
            self.augip_initialized = True
        else:
            
            p = self.augip["p"]
            x = self.augip["x"]
            # Modify bounds according to current solution and gamma
            for i in range(self.n):
                for j in range(self.t):
                    ub = floor( min(self.graver_complexity*gamma, self.u[i][j] - self.current_solution[i][j]) / gamma)
                    lb = ceil( max(-self.graver_complexity*gamma, self.l[i][j] - self.current_solution[i][j]) / gamma)
                    # print "setting lb/ub i,j", i,j, "to", lb, ub
                    p.set_max(x[i,j],ub)
                    p.set_min(x[i,j],lb)
        
        
        if timelimit and self.solver != "Coin":
            p.solver_parameter("timelimit", timelimit)
        
        #start = timing.micros()
        exception = False
        try:
            p.solve()
            
        except MIPSolverException:
            exception = True
        #end = timing.micros()
        #self.fulllog["iterations"][self.iteration]["gammas"][gamma]["solve_time"] = float((end-start)/10**6)
        self.fulllog["iterations"][self.iteration]["gammas"][gamma]["solve_success"] = True
        if exception:
            self.fulllog["iterations"][self.iteration]["gammas"][gamma]["solve_success"] = False
            return vector((0,)*(self.n*self.t))
        if p.get_objective_value() > -1:
            return vector((0,)*(self.n*self.t))
        
        #for i, v in p.get_values(x).iteritems():
        #    print i, v
        # Construct the augmenting step
        h = []
        for i in range(self.n):
            for j in range(self.t):
                h.append(int(p.get_values(x[i,j])))
           
        hh = vector(h)
        
        return hh
        
        
    def _bricks_to_vector(self,bricks):
        v = []
        for i in range(self.n):           
            v+=bricks[i]
        v = vector(v)
        return v
        
        
    def _vector_to_bricks(self,vect):
        bricks = []
        for i in range(self.n):
            b = vector((0,)*self.t)
            for j in range(self.t):
                b[j]=vect[i*self.t + j]
            bricks.append(b)
        return bricks       
        
    
    def native_solve(self):
        global F_container
        global tau
        global r
        global S_star
        global B
        global loop_counter
        global F_count
        global F_signal
        
        global n
        """
        Solves the given problem with the implemented algorithm.
        """
        
        ### TODO: performance idea -- if next solve takes > 1.1*previous_solve, kill the solver and return the previous best step
        # Requires the following parameter. Not sure what p returns if it runs over the limit.
        #p = MixedIntegerLinearProgram(solver = "GLPK")
        #p.solver_parameter("timelimit", 1)
        
        #self.fulllog["start_time"] = float(timing.micros()/10**6)
        self.NFoldLogging.info('NATIVE SOLVER: ')
        
        #initial feasible solution
        if self.current_solution == None:
            self.current_solution = self.find_init_feasible_solution()
        current_obj = self.w_vector.dot_product(self._bricks_to_vector(self.current_solution))
        self.current_obj = current_obj
        self.filelog.write("("+str(current_obj)+",)\n")
        
        self.iteration = int(0)
        
        if self.current_solution != None:
            while True:
                self.iteration += 1
                self.fulllog["iterations"][self.iteration] = {}
                fl = self.fulllog["iterations"][self.iteration]
                fl["x"] = [sage_int_vector_to_tuple(brick) for brick in self.current_solution]
                fl["gammas"] = {}
                #start = timing.micros()
                
                step = self.find_graverbest_step()
                
                #end = timing.micros()
                fl["g"] = sage_int_vector_to_tuple(step)
                #fl["time"] = float( (end-start)/10**6)
                #self.NFoldLogging.warning('time of find_graver_best_step: {}'.format(end-start))
                if (step == 0) or (self.w_vector.dot_product(step) > -1):
                    #self.filelog.write("("+str(self.w_vector.dot_product(self._bricks_to_vector(self.current_solution)))+","+str(end-start)+")")
                    break
                self.NFoldLogging.info('Previous solution: {}'.format(self.current_solution))
                previous_obj = self.w_vector.dot_product(self._bricks_to_vector(self.current_solution))
                self.current_solution = self._vector_to_bricks(self._bricks_to_vector(self.current_solution) + step)
                current_obj = self.w_vector.dot_product(self._bricks_to_vector(self.current_solution))
                self.current_obj = current_obj
                #filelog.write(str(previous_obj)+"\n")
                self.filelog.write("\n")
                self.NFoldLogging.info('Previous objective: {}, New objective:{}'.format(previous_obj, current_obj))
                self.NFoldLogging.info('New solution: {}'.format(self.current_solution))
            #print("working native solve 1")    
            #filelog.write(str(current_obj)+"\n")
            self.filelog.write("\n")
            self.filelog.close()
            self.NFoldLogging.info('find_good_step called {} times'.format(self.all_find_good_step_counter))
            self.NFoldLogging.info("The result is: {}".format(self.current_solution))     
            
            m = 0
            for i in range(self.n):
                for j in range(self.t):
                    m+=self.current_solution[i][j]*self.w[i][j]
            self.NFoldLogging.warning("Objective value from the Native solver: {}".format(m))
            self.native_solution = m
            #self.fulllog["end_time"] = float(timing.micros()/10**6)
            self.fulllog["native_solution"] = int(self.native_solution)
            
            #only for bribing problem
            #print("Cost of Bribing: " + str(int(self.native_solution))) 
            current_sol = np.array(self.current_solution)
            current_sol = current_sol[:,:-tau].flatten()
            current_sol = np.append(current_sol, n-r)
            current_sol = np.concatenate((current_sol,S_star))
            #print(current_sol)
            #print("working native solve 2")
            #print("loop number is " + str(loop_counter))
            #loop_counter = loop_counter+1
            
            #print("native solution is {} and B is {}".format(self.native_solution,B))
            
            
            if int(self.native_solution) <= B:
                F_container = np.vstack((F_container, current_sol))
                print("Found")
                F_count +=1
                F_signal +=1
                S_star = S_star_generator()
                
            elif int(self.native_solution) > B:
                F_signal +=1
                
            
            #print(F_container)
            
        else:
            self.NFoldLogging.warning('Initial feasible point not found.')
          
            
    def create_An_matrix(self):   
        #initalization of the new matrix A
        A_matrix = [[0 for x in range(self.n*self.t)] for y in range(self.r+self.n*self.s)] 
        
        #n-times copy the matrix D = the 1st self.r lines
        for row in range(self.r):
            for col in range(self.n*self.t):
                A_matrix[row][col] = self.D[row][col%self.t]
        
        #n-times copy the matrix A, but only to the main diagonal
        start_index_row = -self.t
        for row in range(self.n*self.s):
                  
            #change the start_index_row 
            if (row)%self.s == 0:
                start_index_row+= self.t
            
            #through columns    
            for col in range(self.n*self.t):
                if start_index_row <= col < (start_index_row+self.t):
                    #copy
                    A_matrix[row+self.r][col] = self.A[row%self.s][col%self.t]  
        
        #return the new A matrix
        
        #for v in A_matrix:
        #    print v
            
        return A_matrix
   
    def milp_instance(self):
        """
        Prepares and returns instance for GLPK.
        """
        #new A^(n) matrix 
        new_A_matrix = self.create_An_matrix()
        
        #initialization of MILP
        nameOfMilp = "milp"
        milp = MixedIntegerLinearProgram(maximization=False, solver = "GLPK")
        d = milp.new_variable(integer=True, nonnegative=True)
        
        #new format of self.b (only one long vector)
        bb = vector(itertools.chain.from_iterable(self.b))
            
        #adding constraints
        for row in range(len(new_A_matrix)):
            milp.add_constraint(sum(new_A_matrix[row][i]*d[i] for i in range(self.n*self.t)), max = bb[row], min = bb[row])
            
        #adding lower & upper bounds
        ll = vector(itertools.chain.from_iterable(self.l))
        uu = vector(itertools.chain.from_iterable(self.u))
        for i in range (self.r+self.n*self.s):
            milp.add_constraint(d[i], max = uu[i], min = ll[i]) 
        
        #set the objective function, which is w*d
        ww = vector(itertools.chain.from_iterable(self.w))  
        milp.set_objective(sum(ww[i]*d[i] for i in range(self.n*self.t)))
        milp.solver_parameter("timelimit", self.milp_timelimit)
        #how does the MILP looks like
        #milp.show()
        
        return milp

    def glpk_solve_instance(self,milp):
        """
        Solves given MILP instance.
        """    
        
        #print milp.solve()
        #milp.solve()
        self.NFoldLogging.info('Starting solve...')
        #self.glpk_start = timing.micros()
        self.glpk_solution = milp.solve()
        #end = timing.micros()
        #self.NFoldLogging.info('Finished. Solve took time ' + str(end-self.glpk_start) + 'us.')

        self.NFoldLogging.warning('Objective Value from GLPK: {}'.format(self.glpk_solution))
                   
    def glpk_solve(self):
        """
        Creates and solves glpk instance
        """
        self.NFoldLogging.info('I am going to solve the MILP instance by glpk.')
        self.NFoldLogging.info('Constructing the instance now...')
        #start = timing.micros()
        try:
            self.glpk_solve_instance(self.milp_instance())
            self.fulllog["glpk_solution"] = int(self.glpk_solution)
        except MIPSolverException:
            print ("GLPK has not found any solution")
            self.fulllog["glpk_solution"] = False
        #end = timing.micros()
        #self.fulllog["glpk_solve_time"] = float((end-self.glpk_start)/10**6)
        #self.NFoldLogging.info('Construct and solve took time ' + str(end-start) + 'us.')
        #self.fulllog["glpk_construct_and_solve"] = float((end-start)/10**6)
    
    
    def solve(self,solver):
        """
        Search Graver best steps until there are no more Graver best steps.
        """
        if solver == "Native":
            self.native_solve()
                            
        elif solver == "GLPK":
            self.glpk_solve()
            
        else:
            print ("Only Native and GLPK solver available.")
    
    def _give_D2(self):
        """
        Build D2 matrix for finding initial feasible solution.
        """
        Ir1 = identity_matrix(self.r)
        Ir2 = diagonal_matrix(self.r,[-1 for i in range(self.r)])
        
        Zrs = zero_matrix(self.r,self.s)
        
        D2 = matrix( block_matrix([self.D, Ir1, Ir2, Zrs, Zrs], nrows =1,ncols =5))
        return D2
        
        
    def _give_A2(self):
        """
        Build A2 matrix for finding initial feasible solution.
        """
                
        Is1 = identity_matrix(self.s)
        Is2 = diagonal_matrix(self.s,[-1 for i in range(self.s)])
        Zsr = zero_matrix(self.s,self.r)
                
        A2 = matrix( block_matrix([self.A,Zsr,Zsr,Is1,Is2],nrows =1,ncols =5))
        return A2       
        
    @staticmethod
    def _firs_part_vect_init_feas_sol(p):
        if p[0] != float('-inf'):
            return p[0]
        elif p[1]!= float('inf'):
            return p[1]
        else:
            return 0
            
    def pos(self,x):
        """
        When positive, return it. If not, return 0.
        """
        if x>=0:
            return x
        else:
            return 0

    def neg(self,x):
        """
        When negative, return it. If not, return 0.
        """
        if x>=0:
            return 0
        else:
            return x
    
    def _neg_or_pos_to_z(self,v):
        """
        Return feasible solution.
        """
        v1 = vector([self.pos(v[i]) for i in range(len(v))])
        v2 = vector([-self.neg(v[i]) for i in range(len(v))])
    
        return [v1,v2]
        
    def find_init_feasible_solution(self):
        """
        Creates and solves auxiliary_program.
        """
        return self.solve_auxiliary_program(self.create_auxiliary_program())
            
    def create_auxiliary_program(self):
        """
        Creates aux program, returns istance for finding the initial feasible solution if no init solution has been given with the instance.
        """
        self.NFoldLogging.info('In create_aux_program')
        
        #new matrices
        D2 = self._give_D2()
        A2 = self._give_A2()
        self.NFoldLogging.info('Has new D = {}, and new A = {}'.format(D2,A2))
        
        #bounds
        max_in_b = 0 #and b should not contain infinity
        for vect in self.b:
            for v in vect:
                if max_in_b<abs(v):
                    max_in_b = abs(v)
           
        l22 = [self.l[0][i] for i in range(self.t)]
        for i in range((2*self.s) + (2*self.r)):
            l22.append(0)
        l2 = [vector((l22))]*self.n
           
        u22 = [self.u[0][i] for i in range(self.t)]
        for i in range((2*self.s) + (2*self.r)):
            u22.append(max_in_b)
        u2 = [vector((u22))]*self.n
        self.NFoldLogging.info('Has new l,u bounds. l = {}, u= {}'.format(l2,u2)) 
        
        #objective function
        w2 = [vector((0,)*self.t + (1,)*((2*self.s) + (2*self.r)))]*self.n 
        self.NFoldLogging.info('Has new objective function: {}'.format(w2))
        
        #first vector in the feasible solution       
        first_part_z = [self._firs_part_vect_init_feas_sol(p) for p in zip (self.l[0], self.u[0])]
        D2_pos = [self.pos(self.b[0][i]) for i in range(len(self.b[0]))]
        D2_neg = [-self.neg(self.b[0][i]) for i in range(len(self.b[0]))]
        A2_pos = [self.pos(self.b[1][i]) for i in range(len(self.b[1]))]
        A2_neg = [-self.neg(self.b[1][i]) for i in range(len(self.b[1]))]        
        z = [vector(list(first_part_z)+ list(D2_pos) + list(D2_neg) + list(A2_pos) +list(A2_neg))]     

        #other vectors in the feasible solution
        for i in range(2,len(self.b)):
            zeros = [0 for y in range(2*self.r+self.t)]
            A2_pos = [self.pos(self.b[i][j]) for j in range(len(self.b[i]))]
            A2_neg = [-self.neg(self.b[i][j]) for j in range(len(self.b[i]))]
            z.append(vector((list(zeros)+list(A2_pos)+list(A2_neg))))
        
        self.NFoldLogging.info('Has the feasible point for the aux. program: {}'.format(z))
            
        #solving the auxiliary program
        auxiliary_program = NFoldIP(A2, D2, self.n, self.b, l2, u2, w2, verbose = logging.DEBUG, graver_complexity= self.graver_complexity, current_solution = z, experimental = self.experimental) 
        
        return auxiliary_program
        
    def _erase_last_2rs_zeros(self,oldv):
        """
        Just cut off the last 2rs zeros.
        """
        final_vector = []
        nev = []
        
        for i in range(self.n):
            control = 0
            for aa in oldv[i]:
                nev.append(aa)
                control+=1
                if(control>=self.t):
                    final_vector+= [vector((nev))]
                    nev = []
                    break 
        return final_vector
        
    def solve_auxiliary_program(self,auxiliary_instance):      
        """
        Solves the given auxiliary instance.  
        """    
        self.NFoldLogging.info('Starting to solve the aux program')  
        auxiliary_instance.solve("Native")
        
        self.NFoldLogging.info('Aux program was solved. Its solution: {}'.format(auxiliary_instance.current_solution))
        
        #check wheather the auxiliary vars are zeros               
        bool_break = 0 
        for j in range(len(auxiliary_instance.current_solution)):
            if bool_break == 1:
                break
            for i in range(self.t,self.t-1+2*self.r+self.s*2):
                if bool_break == 1:
                    break
                if auxiliary_instance.current_solution[j][i] !=0:
                    self.NFoldLogging.warning('No feasible solution found!')
                    bool_break =1
                    break
        
        if bool_break == 0:        
            init_feas_solution = self._erase_last_2rs_zeros(auxiliary_instance.current_solution) 
            print ("init_feas_solution: {}".format(init_feas_solution))       
            return init_feas_solution   
        
        
        else:
            return None
        

In [2]:
import random
from sage.matrix.constructor import matrix
from sage.modules.free_module_element import vector
import numpy as np
import math
import operator as op
from functools import reduce
import timeit
import ctypes, os 
#from nfoldip import *



In [3]:
def run_tests(inst):
    
    global F_container
    global F_signal
    global S_star
    
    A, D, n, b, l, u, w, x, inst_name = inst
    #Delta = int(max((A.numpy().max(), D.numpy().max())))
    #dimension = n*D.ncols() # n*t
    for gc_val in [128]:
        #print("solving for graver_complexity " + str(gc_val))
        try:
            #start = timeit.default_timer()

            ip = NFoldIP(A, D, n, b, l, u, w, verbose = logging.ERROR, graver_complexity=int(gc_val), current_solution = x,
                         experimental="ng1", instancename="inst_name", gamma="log2", solver="GLPK")
            
            ip.native_solve()
            #print("I am here")
            #print(ip.native_solution)
            #stop = timeit.default_timer()
            #print('Time: ', stop - start)
            
        except:
            
            #print("Failed")
            S_star = S_star_generator()
            pass
            



In [4]:


#parse your information here

m = 2                    #number of candidate (excluding the designated candidate)
n = 100               #number of voters including the voters of the designated candidate
#B = 200000              #total bribing budget
EPS = [.5]     #epsilon
p = 0.3                 #probability to get selected in committee
min_bcost = 50          #minimum bribe cost for a voter
max_bcost = 100         #maximmum bribe cost for a voter
rnd_seed = 0
loop_counter = 1
#function to calculate the combination nCr formula
def ncr(n, r):
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer // denom

def bribing_instance_generator(min_bcost,max_bcost):
     
    votes = np.zeros((n,m+1))
    vote_count = np.sum(votes, axis = 0)

    while np.amax(vote_count, axis = 0) == vote_count[m]:
        votes = np.zeros((n,m+1))
        bribe_cost = np.zeros((n,m))
        for i in range(n):
            rnd_vote = random.randint(0, m)
            votes[i,rnd_vote] = 1
            if rnd_vote != m:
                rnd_cost = random.randint(min_bcost, max_bcost)
                bribe_cost[i,rnd_vote] = rnd_cost
            
        vote_count = np.sum(votes, axis = 0)
            
    lamda = np.zeros((int(max(vote_count)+1),m))
    for i in range(m):
        temp = bribe_cost[:,i]
        temp = temp.ravel()[np.flatnonzero(temp)]
        temp = np.cumsum(temp)
        lamda[1:len(temp)+1,i] = temp  
    
    return vote_count, lamda        

#vote_count, lamda = bribing_instance_generator(min_bcost,max_bcost)
#max_V_i = lamda.shape[0] #maximun votes count
#tau = int(np.floor(1/(np.log(1 + eps/4)))) 
#F_container = np.zeros((m*max_V_i + 1 + tau),dtype =int )




#This function will generate unique value for s_star in ascending manner
def S_star_generator():
    
    global tau
    global n
    global rnd_seed
    
    np.random.seed(rnd_seed)
    rnd_seed = rnd_seed + 1
    S_star = np.sort(np.random.choice(range(1,n), size=(tau)))
    while len(S_star) != len(np.unique(S_star)):
        S_star = np.sort(np.random.choice(range(1,n), size=(tau)))
    #print(S_star)
    return S_star

#S_star = S_star_generator()

#Some Auxiliary function
def small_phi(y_i, t):
    
    global p
    
    if t <= y_i:
        return ncr(y_i,t)*math.pow(p,t)*math.pow(1-p,y_i-t)
    else:
        return 0

def big_phi(k, S, t):
    
    global p
    
    ret = 0
    for h in range(S[t]+1):
        ret += small_phi(k,h)
    
    return ret

def big_phi2(k, t):
    
    global p
    
    ret = 0
    for h in range(t+1):
        ret += small_phi(k,h)
    
    return ret

def fkt_matrix_generator(S_star):
    
    global max_V_i
    global tau
    global m
    global n
    
    fkt = np.zeros((tau,max_V_i))
    for k in range(0,max_V_i):
        
        for t in range(0,tau):
            ret = big_phi(k, S_star, t)                
            fkt[t,k] = np.floor(m*n*tau*np.log(ret))
            #print("when k is " + str(k) + ", t is " + str(t) + ", and S[t] is " + str(S_star[t]) +", then big_phi = " + str(ret) + ", and fkt = " + str(fkt[t,k]))
                        
    return fkt

#fkt_matrix = fkt_matrix_generator(S_star)
          

In [5]:
def main_Algo(instance):
    global B
    global m
    global n
    global EPS
    global tau
    global min_bcost
    global max_bcost
    global p
    global max_V_i
    global F_container
    global r
    global S_star
    global F_count
    global F_signal
    
    F_count = 0
    lim = 6
    F_signal = 0
    
    for eps in EPS:
        
        #for i in range(1,26):
        start_overall = timeit.default_timer()
        print("Currently at n = {}, m = {}, p = {}, instance {} with eps {}".format(n,m+1,p,instance,eps))
        time = 0
        cnt = 0

        #instance generate here
        lamda = np.loadtxt("Converted_Data/data_n_{}_m_{}_p_0.{}/{}.txt".format(n,m+1,int(p*10),instance))
        lamda = lamda.astype(int)
        
        
        
        B = int(np.loadtxt("Converted_Data/data_n_{}_m_{}_p_0.{}/B_{}.txt".format(n,m+1,int(p*10),instance)))
        #print("B = " + str(B))
        #p = .9
        
        vote_count = np.count_nonzero(lamda,0)
        vote_count = list(vote_count)
        vote_count.append(n-sum(vote_count))
        '''
        for i in range(m):
            temp_lamda = lamda[:,1][::] 
        lamda[:,1] = lamda[::-1,1]
        print(lamda)
        #print(vote_count)
        '''
        max_V_i = lamda.shape[0] #maximun votes count
        
        tau = int(np.floor(1/(np.log(1 + eps/4)))) 
        F_container = np.zeros((m*max_V_i + 1 + tau),dtype =int ) #F_container generate

        wt = np.transpose(lamda).astype(int)
        m_by_tau = np.zeros((m,tau), dtype = int)
        wt = np.concatenate((wt,m_by_tau), axis = 1)



        for r in range (n-min(int(B/(min_bcost)),n-int(vote_count[-1]), int(n/2)),n+1):
            S_star = S_star_generator()
            F_signal = 0
            if F_count < lim:
                print("Working on {} person".format(n-r))
                #for count in range(star):
                while (F_signal < 2):
                    

                    fkt_matrix = fkt_matrix_generator(S_star)

                    b = [int(n-r)]
                    #tau_by_2b = []
                    for t in range(tau):
                        b.append(-1*(int(np.floor(m*n*tau*np.log((t+1)/tau)))))
                        #tau_by_2b.append(-1*(int(np.floor(m*n*tau*np.log((t+1)/tau)))))
                    #tau_by_2b_copy = tau_by_2b.copy()
                    tau_by_tau = np.identity(tau, dtype =int)

                    b = [b]
                    A = list(np.ones((max_V_i)).astype(int))
                    D = list(np.arange(max_V_i))
                    for t in range(tau):
                        D.append(0)
                        A.append(0)
                    tau_by_tau = np.identity(tau, dtype =int)
                    D1 = np.concatenate((fkt_matrix, tau_by_tau), axis = 1)            
                    D = np.vstack((D,D1)).astype(int)



                    lb = np.transpose(np.zeros((max_V_i,m))).astype(int)
                    lb_slack = -100000*np.ones((m, tau),dtype = int)
                    lb = np.concatenate((lb,lb_slack), axis = 1)
                    ub = np.transpose((lamda > 0) *1).astype(int)
                    ub_slack = 100000*np.ones((m, tau),dtype = int)
                    ub = np.concatenate((ub,ub_slack), axis = 1)

                    #init x generate here
                    init_x = np.zeros((m, max_V_i), dtype = int)
                    tau_by_2b = []
                    for t in range(tau):
                        #b.append(-1*(int(np.floor(m*n*tau*np.log((t+1)/tau)))))
                        tau_by_2b.append(-1*(int(np.floor(m*n*tau*np.log((t+1)/tau)))))
                    tau_by_2b_copy = tau_by_2b.copy()
                    for i in range(m-1):
                        tau_by_2b = np.vstack((tau_by_2b,tau_by_2b_copy)).astype(int)
                    init_x = np.concatenate((init_x,tau_by_2b), axis =1)
                    f2 = b[0][0]
                    f1 = []
                    for i in range(m):
                        f1.append(min(vote_count[i], f2))
                        f2 = f2 - f1[i]
                        init_x[i,int(f1[i])] = 1
                        b.append([1])
                        ub[i,0] = 1

                        for j in range(tau):
                            init_x[i,j-tau] = init_x[i,j-tau] - D[j+1,int(f1[i])]

                    lb = [vector(ll) for ll in lb]
                    ub = [vector(uu) for uu in ub]
                    wt = [vector(ww) for ww in wt]

                    init_x = [vector(xx) for xx in init_x]
                    #A = matrix(A)
                    #print(A)
                    #print(D)
                    ip_instance = (matrix(A), matrix(D), m, b, lb, ub, wt, init_x, 'nfold_voting')
                    #print(matrix(D))
                    #print(ip_instance)
                    #break

                    #print("Number of voters to be bribed: " +str(n-r))
                    start = timeit.default_timer()



                    run_tests(ip_instance)

                    stop = timeit.default_timer()
                    time = time + (stop - start)
                    cnt = cnt + 1
                
            else:
                r = n
                break
        #print("Average time: " +str(time/cnt))
        #calculate the last column to find the best result
        try:
        #print(vote_count)
        #print('Average Time: ', time/cnt)
            avg_time = (time/cnt) 

            temp = np.zeros((len(F_container),n+1),dtype=int)
            #print(temp)
            #print(F_container)
            F_container = np.concatenate([F_container,temp],1)

            F_container = F_container[1:,:]

            F_container = F_container.astype(np.float32)
            #print(vote_count)

            for row in range(len(F_container)):
                #print(F_container[row,:])               

                l = int(F_container[row,-1*tau-n-2])+vote_count[-1]    

                for t in range(l+1):
                    zt = 0


                    fp_temp = small_phi(l,t)
                    for i in range(m):
                        k = np.argmax(F_container[row,i*max_V_i:(i+1)*max_V_i])
                        #print(sol_space[i])
                        #for k in range(int(vote_count[i])+1):
                        #print(F_container[row,i*(max_V_i+1)+k])
                        zt += np.log(big_phi2(vote_count[i]-k,t))#* F_container[row,i*(max_V_i)+k]

                    F_container[row,max_V_i*m+tau+1+t] = fp_temp * np.exp(zt)
                    #print(F_container[row,:]) 

                F_container[row,-1] = np.sum(F_container[row,max_V_i*m+tau+1:-1])
            #print(F_container[:,-1])
                #if F_container[row,-1] == 1:
                #    break
            index = np.argmax(F_container[:,-1],0)
            result = F_container[index,:]            
            result = np.concatenate([result,[avg_time]])


            stop_overall = timeit.default_timer()
            overall_time = stop_overall - start_overall
            print ("Total time = {}".format(overall_time))
            result = np.concatenate([result,[overall_time]])
            print(result[-3:])
            np.savetxt("sushi_output/data_n_{}_m_{}_p_0.{}/{}.txt".format(n,m+1,int(p*10),instance),result)

        except:
            print("failed")
            result = 0

for n in [500]:
    for m in [9]:
        for p in [.7]:
            for iter in range(0,1):
                main_Algo(instance = iter)

Currently at n = 500, m = 10, p = 0.700000000000000, instance 0 with eps 0.500000000000000
Working on 250 person
Found
Found
Working on 249 person
Found
Found
Working on 248 person
Found
Found
Total time = 74.25851320000947
[ 1.          3.02358406 74.2585132 ]
